In [9]:
import os

dataset_path = os.path.join('..', 'Datasets')
raw_path = os.path.join(dataset_path, 'Raw')
combined_path = os.path.join(dataset_path, 'Combined')
cleaned_path = os.path.join(dataset_path, 'Clean')

raw_daily_path = os.path.join(raw_path, 'Daily')
raw_hourly_path = os.path.join(raw_path, 'Hourly')

In [6]:
import pandas as pd
import os

dfs = []

# Load every CSV
for file in os.listdir(raw_daily_path):
    if not file.endswith("csv"):
        continue
    
    full_path = os.path.join(raw_daily_path, file)
    
    dfs.append(pd.read_csv(full_path))

# Concatenate all CSVs together
df = pd.concat(dfs, ignore_index=True)

# Sort by the date
df.sort_values('Date/Time', inplace=True)

# Write out a single CSV
df.to_csv(os.path.join(combined_path, 'daily.csv'))

In [8]:
import pandas as pd
import os

dfs = []

# Load every CSV
for file in os.listdir(raw_hourly_path):
    if not file.endswith("csv"):
        continue
    
    full_path = os.path.join(raw_hourly_path, file)
    
    dfs.append(pd.read_csv(full_path))

# Concatenate all CSVs together
df = pd.concat(dfs, ignore_index=True)

# Sort by the date
df.sort_values('Date/Time (LST)', inplace=True)

# Write out a single CSV
df.to_csv(os.path.join(combined_path, 'hourly.csv'))

In [31]:
import pandas as pd
import os

daily_df = pd.read_csv(os.path.join(combined_path, 'daily.csv'))
hourly_df = pd.read_csv(os.path.join(combined_path, 'hourly.csv'))

rows = []

# Iterate over each day
for daily_df_index, daily_df_row in daily_df.iterrows():
    date = daily_df_row['Date/Time']
    
    year = int(date[0:4])
    month = int(date[5:7])
    day = int(date[8:10])
    
    # Get the rows for this day
    hourly_df_rows = hourly_df[(hourly_df['Year'] == year) & (hourly_df['Month'] == month) & (hourly_df['Day'] == day)]
    
    # Get all weather values for this day
    weather_values = hourly_df_rows['Weather'].values
    
    has_precipitation = False
    
    # Check if any of the weather values have precipitation
    for weather in weather_values:
        if not type(weather) is str:
            continue
        
        has_precipitation = has_precipitation or ('Drizzle' in weather) or ('Rain' in weather) or ('Thunderstorms' in weather) or ('Snow' in weather) or ('Ice' in weather) or ('Hail' in weather)
    
    rows.append([
        date,
        year,
        month,
        day,
        daily_df_row['Min Temp (°C)'],
        daily_df_row['Max Temp (°C)'],
        daily_df_row['Total Precip (mm)'],
        daily_df_row['Spd of Max Gust (km/h)'],
        hourly_df_rows['Rel Hum (%)'].min(),
        hourly_df_rows['Rel Hum (%)'].max(),
        hourly_df_rows['Wind Spd (km/h)'].min(),
        hourly_df_rows['Wind Spd (km/h)'].max(),
        hourly_df_rows['Visibility (km)'].min(),
        hourly_df_rows['Visibility (km)'].max(),
        hourly_df_rows['Stn Press (kPa)'].min(),
        hourly_df_rows['Stn Press (kPa)'].max(),
        1 if has_precipitation else 0
    ])

# Create a new DataFrame using the rows
new_df = pd.DataFrame(rows[1:], columns=[
    'Date',
    'Year',
    'Month',
    'Day',
    'Min Temp',
    'Max Temp',
    'Total Precipitation',
    'Max Gust',
    'Min Humidity',
    'Max Humidity',
    'Min Wind Speed',
    'Max Wind Speed',
    'Min Visibility',
    'Max Visibility',
    'Min Pressure',
    'Max Pressure',
    'Condition'
])

# Drop all rows with NaN (empty) values
new_df.dropna(inplace=True)

# Write it to a CSV
new_df.to_csv(os.path.join(cleaned_path, 'weather.csv'))

/var/folders/cp/s7x7dby15bndh6j4jxs2qxhh0000gn/T/ipykernel_3241/3941156841.py:5: DtypeWarning: Columns (11,19,21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  hourly_df = pd.read_csv(os.path.join(combined_path, 'hourly.csv'))
